# Classificação e exploração de textos financeiros

- Base de dados: https://www.kaggle.com/yash612/stockmarket-sentiment-dataset
- Equbot: https://equbot.com/

## Importação das bibliotecas

In [ ]:
import spacy
spacy.__version__

In [ ]:
import pandas as pd
import string
import matplotlib.pyplot as plt
import seaborn as sns
import random
import numpy as np
import re

## Carregamento da base de dados

In [ ]:
# https://drive.google.com/file/d/1GZ43OUXRqmiAzZxZ5EZzX2Hcm3VJpX1g/view?usp=sharing
base = pd.read_csv('/content/stock_data.csv')

In [ ]:
base.shape

In [ ]:
base.head()

In [ ]:
base.tail()

In [ ]:
np.unique(base['Sentiment'], return_counts=True)

In [ ]:
sns.countplot(x = base['Sentiment']);

In [ ]:
base.isnull().sum()

## Função para pré-processamento dos textos

In [ ]:
pln = spacy.load('en')
pln

In [ ]:
stop_words = spacy.lang.en.stop_words.STOP_WORDS
print(stop_words)

In [ ]:
string.punctuation

In [ ]:
def preprocessamento(texto):
  texto = texto.lower()
  texto = re.sub(r"@[A-Za-z0-9$-_@.&+]+", ' ', texto)
  texto = re.sub(r"https?://[A-Za-z0-9./]+", ' ', texto)
  texto = re.sub(r" +", ' ', texto)

  documento = pln(texto)
  lista = []
  for token in documento:
    lista.append(token.lemma_)

  lista = [palavra for palavra in lista if palavra not in stop_words and palavra not in string.punctuation]
  lista = ' '.join([str(elemento) for elemento in lista if not elemento.isdigit()])

  return lista

In [ ]:
preprocessamento('I will @test9&+ buy the https://www.iaexpert.academy Apple stock. 9 8 care. caring! car')

## Pré-processamento da base de dados

### Limpeza dos textos

In [ ]:
base.head(10)

In [ ]:
base['Text'] = base['Text'].apply(preprocessamento)

In [ ]:
base.head(10)

## Visualização e exploração

### Número médio de caracteres

In [ ]:
base['tamanho'] = base['Text'].apply(len)
base.head()

In [ ]:
base['tamanho'].describe()

In [ ]:
positivo = base[base['Sentiment'] == 1]

In [ ]:
positivo['tamanho'].describe()

In [ ]:
negativo = base[base['Sentiment'] == -1]

In [ ]:
negativo['tamanho'].describe()

### Nuvem de palavras

In [ ]:
textos_positivos = positivo['Text'].tolist()
len(textos_positivos)

In [ ]:
textos_positivos[0:10]

In [ ]:
textos_positivos_string = ' '.join(textos_positivos)

In [ ]:
textos_positivos_string

In [ ]:
len(textos_positivos_string)

In [ ]:
from wordcloud import WordCloud
plt.figure(figsize=(20,10))
plt.imshow(WordCloud().generate(textos_positivos_string));

In [ ]:
lista_negativo = negativo['Text'].tolist()
textos_negativos_string = ' '.join(lista_negativo)

In [ ]:
len(textos_negativos_string)

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(WordCloud().generate(textos_negativos_string));

### Extração de entidades nomeadas

- Siglas: https://spacy.io/api/annotation#named-entities

In [ ]:
documento = pln(textos_positivos_string)

In [ ]:
from spacy import displacy
displacy.render(documento, style = 'ent', jupyter = True)

In [ ]:
empresas_positivas = []
for entidade in documento.ents:
  if entidade.label_ == 'ORG':
    print(entidade.text, entidade.label_)
    empresas_positivas.append(entidade.text)

In [ ]:
len(empresas_positivas)

In [ ]:
empresas_positivas = set(empresas_positivas)

In [ ]:
len(empresas_positivas)

In [ ]:
print(empresas_positivas)

In [ ]:
empresas_negativas = []
documento = pln(textos_negativos_string)
for entidade in documento.ents:
  if entidade.label_ == 'ORG':
    print(entidade.text, entidade.label_)
    empresas_negativas.append(entidade.text)

In [ ]:
empresas_negativas = set(empresas_negativas)
len(empresas_negativas)

In [ ]:
print(empresas_negativas)

In [ ]:
x = {"A", "B", "C"}
y = {'A', "D", "E"}
x.intersection(y)

In [ ]:
x.difference(y), y.difference(x)

In [ ]:
empresas_positivas_negativas = empresas_positivas.intersection(empresas_negativas)

In [ ]:
len(empresas_positivas_negativas)

In [ ]:
print(empresas_positivas_negativas)

In [ ]:
empresas_somente_positivas = empresas_positivas.difference(empresas_negativas)
len(empresas_somente_positivas)

In [ ]:
print(empresas_somente_positivas)

In [ ]:
empresas_somente_negativas = empresas_negativas.difference(empresas_positivas)
len(empresas_somente_negativas)

In [ ]:
print(empresas_somente_negativas)